<a href="https://colab.research.google.com/github/arjunprakash027/HandcraftedML/blob/main/Reinforcement_Learning/Basics/Reinforce_DRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyvirtualdisplay
!pip install pyglet==1.5.1

In [3]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
!pip install git+https://github.com/ntasfi/PyGame-Learning-Environment.git
!pip install git+https://github.com/simoninithomas/gym-games
!pip install huggingface_hub
!pip install imageio-ffmpeg
!pip install pyyaml==6.0


## Imports

In [87]:
import numpy as np

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Gym
import gymnasium as gym
import gym_pygame

# Hugging Face Hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.
import imageio

from tqdm import trange

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## HF HUB

In [118]:
from huggingface_hub import HfApi, snapshot_download
from huggingface_hub.repocard import metadata_eval_result, metadata_save

from pathlib import Path
import datetime
import json
import imageio

import tempfile

import os

def record_video(env, policy, out_path, max_steps=1000, fps=30):
    """
    Generate a replay video of the agent.
    :param env: Gymnasium or Gym environment (must be created with render_mode='rgb_array')
    :param policy: Agent with a F_pass(state) method returning (action, log_prob)
    :param out_path: Path to save the output video (e.g., 'replay.mp4')
    :param max_steps: Maximum steps to record (safety cap)
    :param fps: Desired frames per second for the output video
    """
    images = []
    state, _ = env.reset()
    img = env.render()
    images.append(img)
    done = False
    steps = 0
    while not done and steps < max_steps:
        action, _ = policy.F_pass(state)
        state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        img = env.render()
        images.append(img)
        steps += 1
    imageio.mimsave(out_path, [np.array(frame) for frame in images], fps=fps)

def push_to_hub(repo_id,
                model,
                hyperparameters,
                eval_env,
                video_fps=30
                ):
  """
  Evaluate, Generate a video and Upload a model to Hugging Face Hub.
  This method does the complete pipeline:
  - It evaluates the model
  - It generates the model card
  - It generates a replay video of the agent
  - It pushes everything to the Hub

  :param repo_id: repo_id: id of the model repository from the Hugging Face Hub
  :param model: the pytorch model we want to save
  :param hyperparameters: training hyperparameters
  :param eval_env: evaluation environment
  :param video_fps: how many frame per seconds to record our video replay
  """

  _, repo_name = repo_id.split("/")
  api = HfApi()

  # Step 1: Create the repo
  repo_url = api.create_repo(
        repo_id=repo_id,
        exist_ok=True,
  )

  with tempfile.TemporaryDirectory() as tmpdirname:
    local_directory = Path(tmpdirname)

    # Step 2: Save the model
    torch.save(model, local_directory / "model.pt")

    # Step 3: Save the hyperparameters to JSON
    # Convert numpy int64 to standard int for JSON serialization
    serializable_hyperparameters = {k: int(v) if isinstance(v, np.int64) else v for k, v in hyperparameters.items()}
    with open(local_directory / "hyperparameters.json", "w") as outfile:
      json.dump(serializable_hyperparameters, outfile)

    # Step 4: Evaluate the model and build JSON
    mean_reward, std_reward = evaluate_agent(eval_env,
                                            hyperparameters["max_t"],
                                            hyperparameters["n_evaluation_episodes"],
                                            model)
    # Get datetime
    eval_datetime = datetime.datetime.now()
    eval_form_datetime = eval_datetime.isoformat()

    evaluate_data = {
          "env_id": hyperparameters["env_id"],
          "mean_reward": mean_reward,
          "n_evaluation_episodes": hyperparameters["n_evaluation_episodes"],
          "eval_datetime": eval_form_datetime,
    }

    # Write a JSON file
    with open(local_directory / "results.json", "w") as outfile:
        json.dump(evaluate_data, outfile)

    # Step 5: Create the model card
    env_name = hyperparameters["env_id"]

    metadata = {}
    metadata["tags"] = [
          env_name,
          "reinforce",
          "reinforcement-learning",
          "custom-implementation",
          "deep-rl-class"
      ]

    # Add metrics
    eval = metadata_eval_result(
        model_pretty_name=repo_name,
        task_pretty_name="reinforcement-learning",
        task_id="reinforcement-learning",
        metrics_pretty_name="mean_reward",
        metrics_id="mean_reward",
        metrics_value=f"{mean_reward:.2f} +/- {std_reward:.2f}",
        dataset_pretty_name=env_name,
        dataset_id=env_name,
      )

    # Merges both dictionaries
    metadata = {**metadata, **eval}

    model_card = f"""
  # **Reinforce** Agent playing **{env_id}**
  This is a trained model of a **Reinforce** agent playing **{env_id}** .
  To learn to use this model and train yours check Unit 4 of the Deep Reinforcement Learning Course: https://huggingface.co/deep-rl-course/unit4/introduction
  """

    readme_path = local_directory / "README.md"
    readme = ""
    if readme_path.exists():
        with readme_path.open("r", encoding="utf8") as f:
          readme = f.read()
    else:
      readme = model_card

    with readme_path.open("w", encoding="utf-8") as f:
      f.write(readme)

    # Save our metrics to Readme metadata
    metadata_save(readme_path, metadata)

    # Step 6: Record a video
    video_path =  local_directory / "replay.mp4"
    record_video(env, model, video_path, video_fps)

    # Step 7. Push everything to the Hub
    api.upload_folder(
          repo_id=repo_id,
          folder_path=local_directory,
          path_in_repo=".",
    )

    print(f"Your model is pushed to the Hub. You can view your model here: {repo_url}")

## The base NN and training algo

In [74]:
class Policy(nn.Module):

  def __init__(
      self,
      state_size: int,
      action_size: int,
      hidden_size: int,
  ):

    super(Policy, self).__init__()
    self.fc1 = nn.Linear(state_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, action_size)

  def forward(
      self,
      state: torch.Tensor,
  ):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.softmax(x, dim=1)
    return x

  def F_pass(
      self,
      state
  ):

    state = torch.from_numpy(state).float().unsqueeze(0).to(device)
    probs = self.forward(state).cpu()
    m = Categorical(probs)
    action = m.sample()
    log_prob = m.log_prob(action)

    return action.item(), log_prob


## Reinforce Algorithm

In [92]:
def collect_experience(
    policy,
    env,
    max_t
):
  saved_log_probs = []
  rewards = []
  state = env.reset()[0]

  for t in range(max_t):

    action, log_prob = policy.F_pass(state)
    saved_log_probs.append(log_prob)

    state, reward, done, _, _ = env.step(action)

    rewards.append(reward)

    if done:
      break

  return saved_log_probs, rewards

def calculate_returns(
    rewards,
    gamma
):

  returns = []
  advantage = 0

  for reward in reversed(rewards):

    advantage = reward + gamma * advantage
    returns.insert(0, advantage)

  return returns

def calculate_policy_loss(
    saved_log_probs,
    returns
):

  policy_losses = []

  for log_prob, return_ in zip(saved_log_probs, returns):

    policy_losses.append(-log_prob * return_)

  return torch.cat(policy_losses).sum()

def reinforce(
    policy,
    optimizer,
    n_training_episodes,
    max_t,
    gamma,
    print_every
):

  scores_deque = deque(maxlen=100)
  scores = []

  progress_bar = trange(1, n_training_episodes + 1, desc="Training", leave=True)

  for i_episode in progress_bar:

    saved_log_probs, rewards = collect_experience(policy, env, max_t)

    scores_deque.append(sum(rewards))
    scores.append(sum(rewards))

    returns = calculate_returns(rewards, gamma)

    eps = np.finfo(np.float32).eps.item()
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    policy_loss = calculate_policy_loss(saved_log_probs, returns)

    optimizer.zero_grad()
    policy_loss.backward()
    optimizer.step()

    progress_bar.set_postfix({
                "Avg100": np.mean(scores_deque),
                "Policy Loss": policy_loss.item()
            })

    # if i_episode % print_every == 0:
    #   print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))

  return scores

In [102]:
def evaluate_agent(env, max_steps, n_eval_episodes, policy):
  """
  Evaluate the agent for ``n_eval_episodes`` episodes and returns average reward and std of reward.
  :param env: The evaluation environment
  :param n_eval_episodes: Number of episode to evaluate the agent
  :param policy: The Reinforce agent
  """
  episode_rewards = []
  for episode in range(n_eval_episodes):
    state = env.reset()[0]
    step = 0
    done = False
    total_rewards_ep = 0

    for step in range(max_steps):
      action, _ = policy.F_pass(state)
      new_state, reward, done, info, _ = env.step(action)
      total_rewards_ep += reward

      if done:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

## Cartpole

In [109]:
env_id = "CartPole-v1"

env = gym.make(env_id, render_mode="rgb_array")
eval_env = gym.make(env_id)

state_size = env.observation_space.shape[0]
action_size = env.action_space.n

print("Sample observation", env.observation_space.sample())
print("Action Space Sample", env.action_space.sample())

state_size, action_size


Sample observation [-4.235665   -0.36911774  0.13449888 -0.66568536]
Action Space Sample 1


(4, np.int64(2))

In [94]:
cartpole_hyperparameters = {
    "h_size": 16,
    "n_training_episodes": 1000,
    "n_evaluation_episodes": 10,
    "max_t": 1000,
    "gamma": 1.0,
    "lr": 1e-2,
    "env_id": env_id,
    "state_space": state_size,
    "action_space": action_size,
}

# Create policy and place it to the device
cartpole_policy = Policy(cartpole_hyperparameters["state_space"], cartpole_hyperparameters["action_space"], cartpole_hyperparameters["h_size"]).to(device)
cartpole_optimizer = optim.Adam(cartpole_policy.parameters(), lr=cartpole_hyperparameters["lr"])

In [95]:
scores = reinforce(cartpole_policy,
                   cartpole_optimizer,
                   cartpole_hyperparameters["n_training_episodes"],
                   cartpole_hyperparameters["max_t"],
                   cartpole_hyperparameters["gamma"],
                   100)

Training: 100%|██████████| 1000/1000 [09:25<00:00,  1.77it/s, Avg100=1e+3, Policy Loss=-7.7]


In [103]:
evaluate_agent(eval_env,
               cartpole_hyperparameters["max_t"],
               cartpole_hyperparameters["n_evaluation_episodes"],
               cartpole_policy)

(np.float64(1000.0), np.float64(0.0))

In [113]:
#with tempfile.TemporaryDirectory() as tmpdirname:
local_directory = Path("/Vids")
video_path =  local_directory / "replay.mp4"
record_video(env, cartpole_policy, video_path, 30)


In [115]:
notebook_login()

In [119]:
repo_id = "Arjunrao/reinforce-cartpole"
push_to_hub(repo_id,
          cartpole_policy, # The model we want to save
          cartpole_hyperparameters, # Hyperparameters
          eval_env, # Evaluation environment
          video_fps=30
    )

model.pt:   0%|          | 0.00/3.39k [00:00<?, ?B/s]

Your model is pushed to the Hub. You can view your model here: https://huggingface.co/Arjunrao/reinforce-cartpole
